# Binary Numbers

Lexer, parser

| Syntactic rules| Semantic Rules |
|:-:|:-:|
| sec -> sec_1 `.` sec_2 | sec_1.v + sec_2.v / 2 ^ sec_2.lon |
| sec -> sec_1 digit | sec_1.v *  2 + dig.v, sec.lon = sec_1.lon + 1 |
| sec -> digit | sec.v = digit.v, sec.long = 1 |
| digit| digit.v = digit.lexval[01]

In [1]:
y = 3